In [826]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gc
%matplotlib inline
gc.collect()

4307

In [827]:
import os
Path = "data"
filelist = os.listdir(Path)

In [828]:
#Arquivo JSON
arquivo_json = open('data/00000337320138260506.json','r')
dados_json = json.load(arquivo_json)

In [829]:
#Grandes categorias
numero = dados_json['numero']
processos = dados_json['processos']
publicacoes = dados_json['publicacoes']

In [847]:
class Prep:
    
    def __init__(self,numero,processos,publicacoes):
        self.numero = numero
        self.processos = processos
        self.publicacoes = publicacoes
        
    def prep_num_processo(self):
        proc = self.processos[0]['processo'] #Fonte: andamentos
        id_proc = np.array([self.numero]) #Fonte: cabeçalho
        
        #Numero do processo - Fonte: cabeçalho
        df_id = pd.DataFrame(id_proc,columns=['id'])
        
        #Quebra do numero do processo - Fonte: andamentos
        p = proc.split('.')

        num = int(p[0].split('-')[0]+p[0].split('-')[1])
        ano_inicio = int(p[1])
        ramo = int(p[2])
        tribunal = int(p[3])
        vara_orig = int(p[4])

        arr = np.array([[num, ano_inicio, ramo, tribunal, vara_orig]])

        df_proc = pd.DataFrame(arr,columns=['num_seq','ano_inicio','ramo','tribunal','vara_orig'])
        
        #Concatenacao
        df = pd.concat([df_id, df_proc],axis=1)
        
        return df

In [848]:
prep = Prep(numero, processos, publicacoes)

# Processos

In [849]:
arr = np.array([prep.numero])
df_id = pd.DataFrame(arr,columns=['id'])
df_id

,id
0,0000033-73.2013.8.26.0506


In [850]:
#def prep_num_processo(obj):
#    proc = obj.processos[0]['processo']
#    p = proc.split('.')
#    
#    num = int(p[0].split('-')[0]+p[0].split('-')[1])
#    ano_inicio = int(p[1])
#    ramo = int(p[2])
#    tribunal = int(p[3])
#    vara_orig = int(p[4])
#    
#    arr = np.array([[num, ano_inicio, ramo, tribunal, vara_orig]])
#    
#    df = pd.DataFrame(arr,columns=['num_seq','ano_inicio','ramo','tribunal','vara_orig'])
#    return df


df_processo = prep.prep_num_processo()
df_processo


,id,num_seq,ano_inicio,ramo,tribunal,vara_orig
0,0000033-73.2013.8.26.0506,3373,2013,8,26,506


## Informações Básicas

In [793]:
classe = prep.processos[0]['classe']
classe_area = prep.processos[0]['classe_area']
assunto = prep.processos[0]['assunto']
outros_assuntos = prep.processos[0]['outros_assuntos']
juiz = prep.processos[0]['juiz']
valor = prep.processos[0]['valor']
audiencias = prep.processos[0]['audiencias']
incidentes = prep.processos[0]['audiencias']
total_andamentos = prep.processos[0]['total_andamentos']

df_base = pd.DataFrame(np.array([[classe, classe_area, assunto, outros_assuntos, juiz, audiencias, incidentes,valor,total_andamentos]]), columns=['classe', 'classe_area', 'assunto', 'outros_assuntos', 'juiz', 'audiencias', 'incidentes','valor','total_andamentos'])

In [794]:
df_base

,classe,classe_area,assunto,outros_assuntos,juiz,audiencias,incidentes,valor,total_andamentos
0,Procedimento Comum,Cível,Contratos Bancários,None,Francisco Camara Marques Pereira,None,None,"R$ 52.741,64",95


## Partes

In [735]:
#Parte 0
#partes_tipo_0 = processos[0]['partes'][0]['tipo']
#partes_nome_0 = processos[0]['partes'][0]['nome']
#Parte 1
#partes_tipo_1 = processos[0]['partes'][1]['tipo']
#partes_nome_1 = processos[0]['partes'][1]['nome']

a=np.array(processos[0]['partes'])
a.size
out_tipo = []
out_reqte= []
out_reqda = []
for i in range(a.size):
    tipo = processos[0]['partes'][i]['tipo']
    if tipo == 'Reqte':
        out_reqte.append(processos[0]['partes'][i]['nome'])
    elif tipo == 'Reqda':
        out_reqda.append(processos[0]['partes'][i]['nome'])
arr=[[out_reqte, out_reqda]]
df_partes = pd.DataFrame(arr,columns = ['Reqte','Reqda'])
df_partes

,Reqte,Reqda
0,[Itaú Unibanco S. A.],[Renir da Silva Mendes]


## Advogados

In [736]:
#Advogado 0
#advogados_tipo_0 = processos[0]['advogados'][0]['tipo']
#advogados_nome_0 = processos[0]['advogados'][0]['nome']
#Advogado 1 
#advogados_tipo_1 = processos[0]['advogados'][1]['tipo']
#advogados_nome_1 = processos[0]['advogados'][1]['nome']

In [737]:
a=np.array(processos[0]['advogados'])
a.size
out_advogados = []
for i in range(a.size):
    out_advogados.append(processos[0]['advogados'][i]['nome'])
arr=[[out_advogados]]
df_advogados = pd.DataFrame(arr,columns = ['Advogados'])
df_advogados

,Advogados
0,"[Jorge Donizeti Sanchez, Viviane Zerbinatti de..."


## Andamentos e Anexos

In [738]:
#Andamento t
andamentos_data_0 = processos[0]['andamentos'][0]['data']
andamentos_descricao_0 = processos[0]['andamentos'][0]['descricao']
andamentos_anexo_0 = processos[0]['andamentos'][0]['anexo']
andamentos_anexo_0_label = {}
andamentos_anexo_0_conteudo = {}
#Andamento t-1
andamentos_data_1 = processos[0]['andamentos'][1]['data']
andamentos_descricao_1 = processos[0]['andamentos'][1]['descricao']
andamentos_anexo_1 = processos[0]['andamentos'][1]['anexo']
andamentos_anexo_0_label = {}
andamentos_anexo_0_conteudo = {}
#Andamento t-2
andamentos_data_2 = processos[0]['andamentos'][2]['data']
andamentos_descricao_2 = processos[0]['andamentos'][2]['descricao']
andamentos_anexo_2 = processos[0]['andamentos'][2]['anexo']
andamentos_anexo_2_label = processos[0]['andamentos'][2]['anexo']['label']
andamentos_anexo_2_conteudo = processos[0]['andamentos'][2]['anexo']['conteudo']


In [739]:
a=np.array(processos[0]['andamentos'])
a.size
out_data = []
out_descricao = []
for i in range(a.size):
    out_data.append(processos[0]['andamentos'][i]['data'])
    out_descricao.append(processos[0]['andamentos'][i]['descricao'])
arr=[[out_data,out_descricao]]
df_andamentos = pd.DataFrame(arr,columns = ['dt_andamentos','andamentos'])
df_andamentos

,dt_andamentos,andamentos
0,"[20/09/2017, 19/09/2017, 15/09/2017, 15/09/201...",[Certidão de Publicação Expedida Relação :03...


In [740]:
#a=np.array(processos[0]['andamentos'])
#a.size
#out_data = []
#out_descricao = []
#for i in range(a.size):
#    out_data.append(processos[0]['andamentos'][i]['data'])
#    out_descricao.append(processos[0]['andamentos'][i]['descricao'])
#    out_anexo.append(processos[0]['andamentos'][i]['anexo'])
#df_andamentos = pd.DataFrame(np.array([out_descricao]),columns=out_data)
#df_andamentos

## Petições

In [741]:
#Petição 0
peticoes_data_0 = processos[0]['peticoes'][0]['data']
peticoes_tipo_0 = processos[0]['peticoes'][0]['tipo']
#Petição 1
peticoes_data_1 = processos[0]['peticoes'][1]['data']
peticoes_tipo_1 = processos[0]['peticoes'][1]['tipo']

In [742]:
a=np.array(processos[0]['peticoes'])
a.size
out_data = []
out_tipo = []
for i in range(a.size):
    out_data.append(processos[0]['peticoes'][i]['data'])
    out_tipo.append(processos[0]['peticoes'][i]['tipo'])
arr=[[out_data,out_tipo]]
df_peticoes = pd.DataFrame(arr,columns = ['dt_peticoes','peticoes_tipo'])
df_peticoes

,dt_peticoes,peticoes_tipo
0,"[19/02/2013, 23/07/2014, 02/12/2015, 02/12/201...","[Petição Intermediária, Contrarrazões de Apela..."


In [743]:
#a=np.array(processos[0]['peticoes'])
#a.size
#out_data = []
#out_tipo = []
#for i in range(a.size):
#    out_data.append(processos[0]['peticoes'][i]['data'])
#    out_tipo.append(processos[0]['peticoes'][i]['tipo'])
#df_peticoes = pd.DataFrame(np.array([out_tipo]),columns=out_data)
#df_peticoes

# Publicações

In [744]:
#numeroCNJ = publicacoes[0]['numeroCNJ']
#numeroAntigo = publicacoes[0]['numeroAntigo']
#numeroUnificado = publicacoes[0]['numeroUnificado']
numeroInstancia = publicacoes[0]['numeroInstancia']
#anoPublicacao = publicacoes[0]['anoPublicacao']
dataPublicacao = publicacoes[0]['dataPublicacao']
#diario = publicacoes[0]['diario']
#diarioUF = publicacoes[0]['diarioUF']
#cidadeComarcaDescricao = publicacoes[0]['cidadeComarcaDescricao']
varaDescricao = publicacoes[0]['varaDescricao']
arquivada = publicacoes[0]['arquivada']
complemento = publicacoes[0]['complemento']
#Conteudo
conteudo = publicacoes[0]['conteudo']
despacho = publicacoes[0]['despacho']
#Empresa
#codVinculo = publicacoes[0]['codVinculo']
#vinculo = publicacoes[0]['vinculo']
#Busca
codTermo = publicacoes[0]['codTermo']
termoEncontrado = publicacoes[0]['termoEncontrado']
#buscaLote = publicacoes[0]['buscaLote']
buscaLoteAno = publicacoes[0]['buscaLoteAno']
buscaLoteMes = publicacoes[0]['buscaLoteMes']
#buscaLoteGrupo = publicacoes[0]['buscaLoteGrupo']
#Correcoes
#corrigido = publicacoes[0]['corrigido']
#corrigidoCidade = publicacoes[0]['corrigidoCidade']
#corrigidoConteudo = publicacoes[0]['corrigidoConteudo']
#corrigidoDespacho = publicacoes[0]['corrigidoDespacho']
#corrigidoNumero = publicacoes[0]['corrigidoNumero']
#corrigidoOrgao = publicacoes[0]['corrigidoOrgao']
#corrigidoVara = publicacoes[0]['corrigidoVara']
#conferido = publicacoes[0]['conferido']

In [745]:
a=np.array(publicacoes)
a.size
out_data = []
out_numeroInstancia = []
out_conteudo = []
for i in range(a.size):
    out_data.append(publicacoes[i]['dataPublicacao'])
    out_numeroInstancia.append(publicacoes[i]['numeroInstancia'])
    out_conteudo.append(publicacoes[i]['conteudo'])
    
arr=[[out_data,out_numeroInstancia, out_conteudo]]
df_publicacoes = pd.DataFrame(arr,columns = ['dt_peticoes','numeroInstancia','conteudo'])
df_publicacoes

,dt_peticoes,numeroInstancia,conteudo
0,"[Dec 12 2014 12:00:00:000AM, Jun 12 2015 12:00...","[2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1]",[Nº 0000033-73.2013.8.26.0506- Apelação-Ribeir...


In [702]:
#a=np.array(publicacoes)
#a.size
#out_data = []
#out_numeroInstancia = []
#out_conteudo = []
#out_termoEncontrado = []
#for i in range(a.size):
#    out_data.append(publicacoes[i]['dataPublicacao'])
#    out_numeroInstancia.append(publicacoes[i]['numeroInstancia'])
#    out_conteudo.append(publicacoes[i]['conteudo'])
#    out_termoEncontrado.append(publicacoes[i]['termoEncontrado'])
#df_publicacao = pd.DataFrame(np.array([out_numeroInstancia,out_termoEncontrado, out_conteudo]),columns=out_data)
#df_publicacao
#arr=np.array([[numeroCNJ, numeroAntigo,numeroUnificado,numeroInstancia, anoPublicacao, dataPublicacao, diario,diarioUF, cidadeComarcaDescricao, varaDescricao,arquivada,complemento, conteudo, despacho, codVinculo,vinculo, codTermo, termoEncontrado,buscaLote,buscaLoteAno,buscaLoteMes,buscaLoteGrupo,corrigido,corrigidoCidade,corrigidoConteudo,corrigidoDespacho,corrigidoNumero,corrigidoOrgao,corrigidoVara,conferido]])
#col=['numeroCNJ', 'numeroAntigo','numeroUnificado','numeroInstancia', 'anoPublicacao', 'dataPublicacao', 'diario', 'diarioUF', 'cidadeComarcaDescricao', 'varaDescricao','arquivada','complemento', 'conteudo', 'despacho', 'codVinculo','vinculo', 'codTermo', 'termoEncontrado','buscaLote','buscaLoteAno','buscaLoteMes','buscaLoteGrupo','corrigido','corrigidoCidade','corrigidoConteudo','corrigidoDespacho','corrigidoNumero','corrigidoOrgao','corrigidoVara','conferido'] 

#df_publicacoes = pd.DataFrame (arr,columns=col)   
#df

# Data Frame

In [747]:
df = pd.concat([df_id,df_processo,df_base,df_partes,df_advogados,df_andamentos,df_peticoes],axis=1)
df

,id,num_seq,ano_inicio,ramo,tribunal,vara_orig,classe,classe_area,assunto,outros_assuntos,...,incidentes,valor,total_andamentos,Reqte,Reqda,Advogados,dt_andamentos,andamentos,dt_peticoes,peticoes_tipo
0,0000033-73.2013.8.26.0506,3373,2013,8,26,506,Procedimento Comum,Cível,Contratos Bancários,None,...,None,"R$ 52.741,64",95,[Itaú Unibanco S. A.],[Renir da Silva Mendes],"[Jorge Donizeti Sanchez, Viviane Zerbinatti de...","[20/09/2017, 19/09/2017, 15/09/2017, 15/09/201...",[Certidão de Publicação Expedida Relação :03...,"[19/02/2013, 23/07/2014, 02/12/2015, 02/12/201...","[Petição Intermediária, Contrarrazões de Apela..."
